<a href="https://colab.research.google.com/github/kgpark88/energy-bigdata-analysis/blob/main/spark_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark DataFrame


## Spark 설치

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz 
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

findspark.init()
findspark.find()

'/content/spark-3.1.2-bin-hadoop2.7'

## Creating a DataFrame

First we need to start a SparkSession:

In [2]:
from pyspark.sql import SparkSession

Then start the SparkSession

In [3]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()


You will first need to get the data from a file (or connect to a large distributed file like HDFS, we'll talk about this later once we move to larger datasets on AWS EC2).

In [ ]:
df = spark.read.json("/content/spark-3.1.2-bin-hadoop2.7/examples/src/main/resources/people.json")

#### Showing the data

In [ ]:
# Note how data is missing!
df.show()

In [ ]:
df.printSchema()

In [ ]:
df.columns

In [ ]:
df.describe()

Some data types make it easier to infer schema (like tabular formats such as csv which we will show later). 

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [ ]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Next we need to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [ ]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [ ]:
final_struc = StructType(fields=data_schema)

In [ ]:
df = spark.read.json("/content/spark-3.1.2-bin-hadoop2.7/examples/src/main/resources/people.json", schema=final_struc)

In [ ]:
df.printSchema()

### Grabbing the data

In [ ]:
df['age']

In [ ]:
type(df['age'])

In [ ]:
df.select('age')

In [ ]:
type(df.select('age'))

In [ ]:
df.select('age').show()

In [ ]:
# Returns list of Row objects
df.head(2)

Multiple Columns:

In [ ]:
df.select(['age','name'])

In [ ]:
df.select(['age','name']).show()

### Creating new columns

In [ ]:
# Adding a new column with a simple copy
df.withColumn('newage',df['age']).show()

In [ ]:
df.show()

In [ ]:
# Simple Rename
df.withColumnRenamed('age','supernewage').show()

More complicated operations to create new columns

In [ ]:
df.withColumn('doubleage',df['age']*2).show()

In [ ]:
df.withColumn('add_one_age',df['age']+1).show()

In [ ]:
df.withColumn('half_age',df['age']/2).show()

In [ ]:
df.withColumn('half_age',df['age']/2)

We'll discuss much more complicated operations later on!

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [ ]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [ ]:
sql_results = spark.sql("SELECT * FROM people")

In [ ]:
sql_results

In [ ]:
sql_results.show()

In [ ]:
spark.sql("SELECT * FROM people WHERE age=30").show()